In [1]:

import json
import sys
import os
import onnx
import onnx2keras
import tempfile
import numpy as np
import tensorflow as tf


In [15]:

from tensorflow.python.framework import graph_util

In [90]:
# model_file = "/bench_data/SimpleLogisticReg/model.onnx"
# model_file = "/bench_data/image_cls/densenet161.onnx"
model_file = "/bench_data/SimpleFFNN/model.onnx"
model_onnx = onnx.load(model_file)

In [91]:
assert len(model_onnx.graph.input) == 1

input_names = [input_.name for input_ in model_onnx.graph.input]

In [92]:
onnx_model = model_onnx

for i in range(len(onnx_model.graph.node)):
	for j in range(len(onnx_model.graph.node[i].input)):
		if ":" in onnx_model.graph.node[i].input[j]:
			onnx_model.graph.node[i].input[j] = onnx_model.graph.node[i].input[j].replace(":", "_")

	for j in range(len(onnx_model.graph.node[i].output)):
		if ":" in onnx_model.graph.node[i].output[j]:
			onnx_model.graph.node[i].output[j] = onnx_model.graph.node[i].output[j].replace(":", "_")

for i in range(len(onnx_model.graph.input)):
	if ":" in onnx_model.graph.input[i].name:
		onnx_model.graph.input[i].name = onnx_model.graph.input[i].name.replace(":", "_")

for i in range(len(onnx_model.graph.output)):
	if ":" in  onnx_model.graph.output[i].name:
		onnx_model.graph.output[i].name = onnx_model.graph.output[i].name.replace(":", "_")

for i in range(len(onnx_model.graph.initializer)):
	if ":" in  onnx_model.graph.initializer[i].name:
		onnx_model.graph.initializer[i].name = onnx_model.graph.initializer[i].name.replace(":", "_")



In [93]:
onnx_model

ir_version: 7
producer_name: "pytorch"
producer_version: "1.12.1"
graph {
  node {
    input: "onnx__Gemm_0"
    input: "linear1.weight"
    input: "linear1.bias"
    output: "onnx__Relu_7"
    name: "Gemm_0"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "transB"
      i: 1
      type: INT
    }
  }
  node {
    input: "onnx__Relu_7"
    output: "onnx__Gemm_8"
    name: "Relu_1"
    op_type: "Relu"
  }
  node {
    input: "onnx__Gemm_8"
    input: "linear2.weight"
    input: "linear2.bias"
    output: "onnx__Relu_9"
    name: "Gemm_2"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "transB"
      i: 1
      type: INT
    }
  }
  node {
    input: "onnx__Relu_9"
    output: "onnx__Gemm_10"
    name: 

In [94]:

input_names = [input_.name for input_ in model_onnx.graph.input]

In [95]:
input_names

['onnx__Gemm_0']

In [96]:
input_names

['onnx__Gemm_0']

In [100]:
inputs = [input_ for input_ in model_onnx.graph.input]
input_shape = tuple(d.dim_value for d in inputs[0].type.tensor_type.shape.dim)
input_shape

(500, 20)

In [101]:
x = np.random.random(input_shape)
x = np.float32(x)
x

array([[0.06665545, 0.84347326, 0.58189255, ..., 0.437123  , 0.31091574,
        0.5021986 ],
       [0.429636  , 0.45136368, 0.27877587, ..., 0.05679522, 0.54380655,
        0.17073445],
       [0.409653  , 0.00577834, 0.6634853 , ..., 0.15607181, 0.09779806,
        0.3926815 ],
       ...,
       [0.17229171, 0.47023144, 0.66485035, ..., 0.6988369 , 0.3616517 ,
        0.07267076],
       [0.38140118, 0.93936807, 0.8004927 , ..., 0.6854865 , 0.42916557,
        0.6992769 ],
       [0.37052566, 0.12898919, 0.5311173 , ..., 0.8691879 , 0.58729076,
        0.41788802]], dtype=float32)

In [63]:
# model_k = onnx2keras.onnx_to_keras(model_onnx, input_names)

In [64]:

# with tf.compat.v1.Session() as sess:
#     pred = model_k.apply(x)
#     o = sess.run(pred)
#     storage_path = "LogisticReg1"
#     tf.saved_model.save(model_k, storage_path)
# o

In [23]:
# with tf.compat.v1.Session() as sess:
#     pred = model_k.apply(x)
#     sess.run(pred)
import logging
logging.basicConfig(level=logging.INFO)

In [97]:
def fix_missing_tensor_contents(graph_def):
    nodes = graph_def.node
    for x in nodes:
        name = x.name
        dtype = x.attr["dtype"].type
        x_shape = [i.size for i in x.attr["value"].tensor.tensor_shape.dim]
        has_tensor_content = len(x.attr["value"].tensor.tensor_content) > 0
#         print(f"name={name} \t\t dtype={dtype}, x_shape={x_shape} has_tensor_content={has_tensor_content}")
        if len(x_shape) > 0 and not has_tensor_content:
            print(f"Node has shape but no tensor_content: {name}")
            float_vals = x.attr["value"].tensor.float_val
            has_floats = len(float_vals) > 0
            if not has_floats:
                raise RuntimeError(f"No tensor_content and not float_val: {name}")
            print(f"Converting float_val to tensor_content: {name}")
            float_vals = np.array(list(float_vals), dtype="float32")
            arr = np.ndarray(shape=x_shape, dtype="float32", buffer=float_vals)
            tensor_content = numpy_helper.from_array(arr)
            x.attr["value"].tensor.tensor_content = tensor_content.raw_data
            arr_1 = array.array("f", x.attr["value"].tensor.tensor_content)
            arr_1 = np.array(arr_1).reshape(x_shape)
            assert arr_1 == arr
#             del x.attr["value"].tensor.float_val

In [113]:
def rename_output_node(frozen_graph, current_name):
    found = False
    for n in frozen_graph.node:
        if n.name == current_name:
            found = True
            break
    assert found
    n.name = "ModelOut"
def rename_input_node(frozen_graph, current_name):
    found = False
    for n in frozen_graph.node:
        if n.name == model_input:
            assert not found
            n.name = "ModelInput"
            found = True
        for i in range(len(n.input)):
            if n.input[i] == model_input:
                n.input[i] = "ModelInput"

In [118]:
with tf.compat.v1.Session() as sess:
    model_k = onnx2keras.onnx_to_keras(model_onnx, input_names, verbose=False)
    import tf_encrypted as tfe
    with tfe.protocol.SecureNN():
        tfe_model = tfe.keras.models.clone_model(model_k)
    pred = model_k.apply(x)
    storage_path = "LogisticReg1/saved_model-1.pb"
#     storage_path = "densenet161/saved_model.pb"
    model_output = model_k.output.name.replace(":0", "")
    model_input = model_k.input.name.replace(":0", "")
    constant_graph = graph_util.convert_variables_to_constants(
        sess, sess.graph.as_graph_def(), [model_output]
    )
    frozen_graph = graph_util.remove_training_nodes(constant_graph)
    fix_missing_tensor_contents(frozen_graph)
    rename_output_node(frozen_graph, model_output)
    rename_input_node(frozen_graph, model_input)
    with open(storage_path, "wb") as f:
        f.write(frozen_graph.SerializeToString())


INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> onnx__Gemm_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> 12.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight linear1.weight with shape (10, 20).
DEBUG:onnx2keras:Found weight linear1.bias with shape (10,).
DEBUG:onnx2keras:Found weight linear2.weight with shape (5, 10).
DEBUG:onnx2keras:Found weight linear2.bias with shape (5,).
DEBUG:onnx2keras:Found weight linear3.weight with shape (1, 5).
DEBUG:onnx2keras:Found weight linear3.bias with shape (1,).
DEBUG:onnx2keras:Found input onnx__Gemm_0 with shape [20]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Gemm
DEBUG:onnx2keras:node_name: onnx__Relu_7
DEBUG:onnx2keras:node_params: {'alpha': 1.0, 'beta': 1.0, 'transB': 1, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx

RuntimeError: TF Encrypted does not yet support the InputLayer layer.

FailedPreconditionError: Error while reading resource variable onnx__Sigmoid_11_3/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/onnx__Sigmoid_11_3/kernel/N10tensorflow3VarE does not exist.
	 [[node onnx__Sigmoid_11_3/kernel/Read/ReadVariableOp (defined at /home/amin/miniconda3/envs/ma5/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'onnx__Sigmoid_11_3/kernel/Read/ReadVariableOp':
  File "/home/amin/miniconda3/envs/ma5/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/amin/miniconda3/envs/ma5/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/amin/miniconda3/envs/ma5/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/amin/miniconda3/envs/ma5/lib/python3.6/site-packages/traitlets/config/application.py", line 664, 

In [115]:
frozen_graph

node {
  name: "ModelInput"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 20
        }
      }
    }
  }
  attr {
    key: "value"
    value {
    }
  }
}
node {
  name: "onnx__Relu_7_3/kernel"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 20
          }
          dim {
            size: 10
          }
        }
        tensor_content: "x\366\021\275\033:\360<k\322\n=\226I\304=\t\273I\275<\203a\275\020#\225\275,\0234=rM\010\276#\351\252\274\222\301p=\320\017\000\276\3534\311\275\326t2\274\021S\202\276%;\251>\276^5<\246\373\n=\366\216\234\276\305\303\273>\366\200\260\271x`K>\251\236\364=\327\202\033>\273\033\001>\277R1\276\207>X\276W\301\016\276\353u\003

In [106]:
model_input = model_k.input.name.replace(":0", "")

In [107]:
model_input

'onnx__Gemm_0_18'

In [110]:
found = False
for n in frozen_graph.node:
    if n.name == model_input:
        assert not found
        n.name = "ModelInput"
        found = True
    for i in range(len(n.input)):
        if n.input[i] == model_input:
            n.input[i] = "ModelInput"
frozen_graph

node {
  name: "ModelInput"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 20
        }
      }
    }
  }
  attr {
    key: "value"
    value {
    }
  }
}
node {
  name: "onnx__Relu_7_1/kernel"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 20
          }
          dim {
            size: 10
          }
        }
        tensor_content: "x\366\021\275\033:\360<k\322\n=\226I\304=\t\273I\275<\203a\275\020#\225\275,\0234=rM\010\276#\351\252\274\222\301p=\320\017\000\276\3534\311\275\326t2\274\021S\202\276%;\251>\276^5<\246\373\n=\366\216\234\276\305\303\273>\366\200\260\271x`K>\251\236\364=\327\202\033>\273\033\001>\277R1\276\207>X\276W\301\016\276\353u\003

In [89]:
for layer in model_k.layers:
    print(layer.name)
    print(layer.get_output_at(0).get_shape().as_list())

onnx__Gemm_0
[None, 2]
onnx__Sigmoid_3
[None, 1]
4
[None, 1]


In [87]:
model_k.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
onnx__Gemm_0 (InputLayer)    [(None, 2)]               0         
_________________________________________________________________
onnx__Sigmoid_3 (Dense)      (None, 1)                 3         
_________________________________________________________________
4 (Activation)               (None, 1)                 0         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [52]:
tensor_content.raw_data

b'\x1c\xb3y\xbf'

In [55]:
import array

In [57]:
array.array("f", tensor_content.raw_data)

array('f', [-0.9753892421722412])

In [120]:
a = np.random.random((2,3))

In [121]:
a

array([[0.3677117 , 0.13842357, 0.83730658],
       [0.7248403 , 0.45039734, 0.6488238 ]])

In [122]:
a[0][0]

0.36771169514945035

In [123]:
a[0][0] = 1.0

In [124]:
a

array([[1.        , 0.13842357, 0.83730658],
       [0.7248403 , 0.45039734, 0.6488238 ]])